In [1]:
import nltk
import os
import functools

from collections import defaultdict

import numpy as np

In [10]:
set(list(zip(*raters[1]))[0]) - set(list(zip(*raters[0]))[0])

{'07_01_Relational_Design_Overview_Part_4.txt',
 '07_02_Functional_Dependencies_Part_2.txt',
 '07_02_Functional_Dependencies_Part_4.txt',
 '07_03_Boyce_Codd_Normal_Form_Part_2.txt',
 '07_03_Boyce_Codd_Normal_Form_Part_3.txt',
 '07_04_Multivalued_Dependencies_and_4th_Normal_Form_Part_2.txt',
 '07_04_Multivalued_Dependencies_and_4th_Normal_Form_Part_4.txt',
 '07_05_Shortcomings_of_BCNF_and_4NF.txt',
 '13_05_Materialized_Views_Part_2.txt',
 '13_05_Materialized_Views_Part_3.txt',
 '14_01_Authorization_Part_4.txt'}

In [2]:
stemmer = nltk.PorterStemmer()
ngrams = defaultdict(set)
raters = []

for name in os.listdir('../../../data/keywords/'):
    with open('../../../data/keywords/' + name) as f:
        rater = {}
        for line in f:
            file, term, _, rank = line.strip().split(',')
            rank = int(rank)
            if rank == 0:
                rank = 6
            tokens = term.lower().split(' ')
            stemmed = ' '.join(stemmer.stem(token) for token in tokens)
            rater[file, stemmed] = rank
            
            if len(tokens) > 1:
                ngrams[file].add(stemmed)
                
        raters.append(rater)
        
            
tfidf_rankings = {}

with open('./wiki_keys.csv') as f:
    for line in f:
        doc, term, rank = line.strip().split(',')
        rank = int(rank)
        tokens = term.lower().split(' ')
        stemmed = ' '.join(stemmer.stem(token) for token in tokens)
        tfidf_rankings[doc, stemmed] = rank
        
        if len(tokens) > 1:
            ngrams[file].add(stemmed)
        
combined_raters = {}
combined_keys = functools.reduce(lambda x, y: x.intersection(y), [set(rater.keys()) for rater in raters])
for key in combined_keys:
    for rater in raters:
        if key in rater:
            if key not in combined_raters:
                combined_raters[key] = rater[key]
            else:
                combined_raters[key] = min(combined_raters[key], rater[key])

for key, val in combined_raters.items():
    if val == 6:
        combined_raters[key] = 0

all_keys = set()

for name in os.listdir('/Users/andrewlamb/Google_Drive/Stanford/CS199/DB_SelfPaced/Flat'):
    with open('/Users/andrewlamb/Google_Drive/Stanford/CS199/DB_SelfPaced/Flat/' + name) as f:
        lines = []
        for line in f:
            line = line.strip('.| \n')

            stemmed = ' '.join([stemmer.stem(token) for token in line.split(' ')])
            lines.append(stemmed)
        document = ' '.join(lines)
        document = document.lower()

        
        for ngram in ngrams[name]:
            document = document.replace(ngram, '')
            all_keys.add((name, ngram))
            
        for token in document.split(' '):
            all_keys.add((name, token))


data = []
for key in all_keys:
    if key in combined_raters:
        rank = combined_raters[key]
        if rank > 0:
            data.append((1, key, 1))
        else:
            data.append((1, key, 2))
    else:
        data.append((1, key, 3))
        
    if key in tfidf_rankings:
        rank = tfidf_rankings[key]
        if rank > 0:
            data.append((2, key, 1))
        else:
            data.append((2, key, 2))
    else:
        data.append((2, key, 3))

In [3]:
# Print a few random data points to spot check
for i in np.random.choice(range(len(data)), size=10, replace=False):
    print(data[i])
    
print('#'*80)

# Print a few that are definitely included in the index
cat_1 = [item for item in data if item[2] == 1]
for i in np.random.choice(range(len(cat_1)), size=10, replace=False):
    print(cat_1[i])
print('#'*80)
    
# Print a few in category 2
cat_2 = [item for item in data if item[2] == 2]
for i in np.random.choice(range(len(cat_2)), size=10, replace=False):
    print(cat_2[i])

(2, ('02_02_Querying_Relational_Databases.txt', 'have'), 3)
(1, ('14_01_Authorization_Part_4.txt', 'separ'), 3)
(1, ('03_01_Well_Formed_XML_Part_2.txt', 'thorugh'), 3)
(1, ('10_01_Indexes_Part_2.txt', 'them'), 3)
(1, ('08_04_XQuery_Demo.txt', 'x'), 3)
(1, ('07_03_Boyce_Codd_Normal_Form_Part_3.txt', '2,'), 3)
(2, ('06_04_Subqueries_in_WHERE_Clause.txt', "didn't"), 3)
(2, ('08_02_XPath_Demo.txt', 'type'), 3)
(2, ('05_02_Set_Operators_Renaming_Notation_Part_1.txt', 'in'), 3)
(2, ('12_01_Motivation_and_Overview.txt', 'use'), 3)
################################################################################
(1, ('05_01_Select_Project_Join_Part_1.txt', 'relat algebra queri languag'), 1)
(2, ('07_04_Multivalued_Dependencies_and_4th_Normal_Form_Part_3.txt', 'student'), 1)
(1, ('07_01_Relational_Design_Overview_Part_2.txt', 'fourth normal form'), 1)
(2, ('07_01_Functional_Dependencies_Part_5.txt', 'relat'), 1)
(2, ('15_02_Basic_Recursive_WITH_Statement_Demo.txt', 'tabl'), 1)
(1, ('08_04_XQuery

In [4]:
t = nltk.AnnotationTask(data=data)
t.kappa()

0.0028326595397245584